<h1 style="color: lightgreen; line-height: 0.5em;">Análise Fundamentalista Mercado de Ações</h1>
<h3>Projeto Ciencia de Dados: Carteira de Ações com Inteligência Articial</h3>

<ul>
<li>Projeto Inspiração: <a href="https://medium.com/swlh/teaching-a-machine-to-trade-stocks-like-warren-buffett-part-i-445849b208c6">
https://medium.com/swlh/teaching-a-machine-to-trade-stocks-like-warren-buffett-part-i-445849b208c6</a></li>
<li>Resumo Fundamentos Empresas BR: <a href="https://fundamentus.com.br/index.php">https://fundamentus.com.br/index.php</a></li>
</ul>


In [226]:
import pandas as pd
import os


In [227]:
empresas = [
    # 'ABEV3',
    "ITUB4"
    # 'ASAI3',
    # 'AZUL4',
    # 'B3SA3',
    # 'BBSE3',
    # 'BEEF3',
    # 'BRAP4',
    # 'BRFS3',
    # 'BRKM5',
    # 'BRML3',
    # 'CCRO3',
    # 'CIEL3',
    # 'CMIG4',
    # 'COGN3',
    # 'CPFE3',
    # 'CPLE6',
    # 'CRFB3',
    # 'CSAN3',
    # 'CSNA3',
    # 'CVCB3',
    # 'CYRE3',
    # 'ECOR3',
    # 'EGIE3',
    # 'ELET3',
    # 'ELET6',
    # 'EMBR3',
    # 'ENBR3',
    # 'ENEV3',
    # 'ENGI11',
    # 'EQTL3',
    # 'EZTC3',
    # 'FLRY3',
    # 'GGBR4',
    # 'GOAU4',
    # 'GOLL4',
    # 'HAPV3',
    # 'HYPE3',
    # 'ITSA4',
    # 'JBSS3',
    # 'JHSF3',
    # 'KLBN11',
    # 'LREN3',
    # 'MGLU3',
    # 'MRFG3',
    # 'MRVE3',
    # 'MULT3',
    # 'NTCO3',
    # 'PCAR3',
    # 'PETR3',
    # 'PETR4',
    # 'PRIO3',
    # 'QUAL3',
    # 'RADL3',
    # 'RAIL3',
    # 'RENT3',
    # 'SBSP3',
    # 'SUZB3',
    # 'TAEE11',
    # 'TIMS3',
    # 'TOTS3',
    # 'UGPA3',
    # 'USIM5',
    # 'VALE3',
    # 'VIVT3',
    # 'WEGE3',
    # "YDUQ3",
]


In [228]:
def starWithUnderScore(dir):
    return dir[-10:-4]


def startWithNotUnderScore(dir):
    return dir[-9:-4]


def formatDataFrame(name, file, sheet_name):
    dataframe = pd.read_excel(f'balancos/{file}', sheet_name=sheet_name)
    dataframe.iloc[0, 0] = name
    dataframe.columns = dataframe.iloc[0]
    dataframe = dataframe[1:]
    dataframe = dataframe.set_index(name)
    return dataframe


handlerStartWith = {"True": starWithUnderScore,
                    "False": startWithNotUnderScore, "dataframe": formatDataFrame}


<p>Criar Dicionários de Dataframes com cada empresas:</p>


In [229]:
fundamentos = {}
files = os.listdir('balancos')
for file in files:
    name = handlerStartWith[str(file[-11:-4].startswith('_'))](file)
    if name in empresas:
        balanco = handlerStartWith['dataframe'](name, file, 0)
        dre = handlerStartWith['dataframe'](name, file, 1)
        fundamentos[name] = pd.concat([balanco, dre])

WARNING *** file size (68522) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (68522) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


<p>Pegar Preços das Ações nas Datas correspondentes</p>


In [230]:
print(fundamentos["ITUB4"].head())

0                                           31/12/2020     30/09/2020  \
ITUB4                                                                   
Ativo Total                              210399002.624  206001995.776   
Ativo Circulante                          20804999.168   10561999.872   
Disponibilidades                                 41000         262000   
Aplicações Interfinanceiras de Liquidez   16584999.936    9009999.872   
Títulos e Valores Mobiliários                   274000         188000   

0                                           30/06/2020     31/03/2020  \
ITUB4                                                                   
Ativo Total                              199988002.816  197917999.104   
Ativo Circulante                          11292000.256   25949999.104   
Disponibilidades                                123000    6070000.128   
Aplicações Interfinanceiras de Liquidez    8835999.744    13628999.68   
Títulos e Valores Mobiliários                   65

In [231]:
df_cotacoes = pd.read_csv("../data/files/cotacoes.csv")

In [232]:
cotacoes = {}
for company in df_cotacoes["Company"].unique():
    cotacoes[company] = df_cotacoes.loc[df_cotacoes["Company"] == company, :]

print(cotacoes["ITUB4"].shape)

(1487, 8)


In [233]:
for empresa in empresas:
    if cotacoes[empresa].isnull().values.any():
        cotacoes.pop(empresa)
        fundamentos.pop(empresa)
empresas = list(cotacoes.keys())

In [234]:
for empresa in fundamentos:
    tabela = fundamentos[empresa].T
    tabela.index = pd.to_datetime(tabela.index, format="%d/%m/%Y")
    tabela_cotacao = cotacoes[empresa].set_index('Date')
    tabela_cotacao = tabela_cotacao[["Adj Close"]]

    tabela = tabela.merge(tabela_cotacao, right_index=True, left_index=True)
    tabela.index.name = empresa
    fundamentos[empresa] = tabela

display(fundamentos["ITUB4"])

,Ativo Total,Ativo Circulante,Disponibilidades,Aplicações Interfinanceiras de Liquidez,Títulos e Valores Mobiliários,Relações Interfinanceiras,Relações Interdependências,Operações de Crédito,Operações de Arrendamento Mercantil,Outros Créditos,...,Receitas,Despesas,Resultado Antes Tributação/Participações,Provisão para IR e Contribuição Social,IR Diferido,Participações/Contribuições Estatutárias,Reversão dos Juros sobre Capital Próprio,Part. de Acionistas Não Controladores,Lucro/Prejuízo do Período,Adj Close
ITUB4,,,,,,,,,,,,,,,,,,,,,


<h4 style="color: orange">Remover da análise tabelas que tem colunas que são diferentes</h4>


In [235]:
# colunas = list(fundamentos["ABEV3"].columns)
# for empresas in fundamentos:
#     if set(colunas) != set(fundamentos[empresa].columns):
#         fundamentos.pop(empresa)
# print(len(fundamentos))


<h4>Ajeitando colunas com o mesmo nomes</h4>


In [236]:
# text_colunas = ";".join(colunas)
# colunas_modificadas = []
# for coluna in colunas:
#     if colunas.count(coluna) == 2 and coluna not in colunas_modificadas:
#         text_colunas = text_colunas.replace(
#             ";" + coluna + ";", ";" + coluna + "_1;", 1)
#     colunas_modificadas.append(coluna)
# colunas = text_colunas.split(";")


In [237]:
# for empresa in fundamentos:
#     print(fundamentos[empresa].shape)
#     if fundamentos[empresa].shape[1] < 42:
#         print(empresa)
#         print(colunas)


In [238]:
# valores_vazios = dict.fromkeys(colunas, 0)
# total_linhas = 0

# for empresa in fundamentos:
#     tabela = fundamentos[empresa]
#     total_linhas += tabela.shape[0]
#     for coluna in colunas:
#         qte_vazios = pd.isnull(tabela[coluna]).sum()
#         valores_vazios[coluna] += qte_vazios


In [239]:
# remover_colunas = []
# for coluna in valores_vazios:
#     if valores_vazios[coluna] > 50:
#         remover_colunas.append(coluna)


# for empresa in fundamentos:
#     fundamentos[empresa] = fundamentos[empresa].drop(remover_colunas, axis=1)
#     fundamentos[empresa] = fundamentos[empresa].ffill()


In [240]:
# for empresa in fundamentos:
#     print(fundamentos[empresa].shape)
